<img src="https://i.imgur.com/SsfgzAq.png" width="50%"/>



In [56]:
%reload_ext sql
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

<b> Query 1 - Hiring Faculty - Program Biology
> <b> Question: </b> Did we hire enough faculty for the program 'Biology' based on the historical enrollment trend?
  <br> <b> Answer: 
* In 2016 we had an overcapacity of 60 students and in 2017 we added 15 additional Faculty members to handle this.
* However we still had an overcapitacity of 16 students in 2017 and we added 4 additional Faculty members in 2018 to absorb the over-capacity.
* For the first semester of 2019 we have a remainder of 247 spots, meaning we did not overload our faculty members and could absorb all the enrollment.
* This means we could downsize the faculty for Biology or re-assign faculty members, after this initial increase in enrollment.

In [57]:
%%sql

SELECT  program_name as 'Program Name', count(FID) as 'Faculty Members' , sum(Act) as 'Actual Enrollment' ,sum(Cap) as Capacity, sum(Rem) as 'Remaining Spots' , YEAR as Year
FROM COURSES_FACT
left JOIN PROGRAMS_DIM USING (PID)
where program_name  like "%Biology%" 
group by program_name , year
order by year desc

 * sqlite:///CourseDataWarehouse.db
Done.


Program Name,Faculty Members,Actual Enrollment,Capacity,Remaining Spots,Year
Biology,76,1346,1593,247,2019
Biology,164,2726,2726,0,2018
Biology,160,2741,2725,-16,2017
Biology,145,2515,2455,-60,2016
Biology,75,1304,1200,-104,2015


<b> Query 2 - Possible Burnout - Faculty for Business Analytics program
> <b> Question: </b> Which faculty members are most likely to get a burnout in the MSBA program, based on their workload?
    <br> <b> Answer: </b> These professors should be sent an e-mail for psychological evaluation and counseling. Especially Prof. Tao and Prof. Huntley.
    <br> However, throughout the years 2015-2019 the workload decreased.

In [58]:
%%sql
SELECT DISTINCT instructor_lname|| ', ' ||instructor_fname as instructor ,COUNT(DISTINCT Title) as Number_of_Preps,COUNT(Title) as Number_of_Sections,sum(act), sum(cap), sum(rem),CatalogYear
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE  program_name like "%information%"  or program_name like "%Business Analytics%" or program_name like "%operation%"
GROUP by CatalogYear ,instructor
HAVING Number_of_Sections> 4
ORDER BY CatalogYear, Number_of_Preps DESC;

 * sqlite:///CourseDataWarehouse.db
Done.


instructor,Number_of_Preps,Number_of_Sections,sum(act),sum(cap),sum(rem),CatalogYear
"Tao, Jie",5,7,121,166,45,2015_2016
"Huntley, Christopher L.",5,7,162,186,24,2015_2016
"Ozcelik, Yasin",4,8,192,212,20,2015_2016
"Lee, Patrick S.",3,6,145,152,7,2015_2016
"Dhanda, Kanwalroop Kathy",2,6,157,173,16,2015_2016
"Sklar, Diane B.",2,5,145,143,-2,2015_2016
"Tao, Jie",5,9,170,221,51,2016_2017
"Huntley, Christopher L.",5,7,128,169,41,2016_2017
"Lee, Patrick S.",3,5,131,131,0,2016_2017
"Vinekar, Vishnu V.",3,6,135,162,27,2016_2017


<b> Query 3 - Popular Programs
> <b> Question: </b> What are the most popular programs in 2018_2019 and did the ranking change respective to the prior year?
    <br> <b> Answer: 
    1. Nursing is still number 1!
    2. English remains at the 2nd place
    3. Biology beats Mathematics in 2018_2019 relative the the year prior. 

In [59]:
%%sql
SELECT program_name as Program, CatalogYear, sum(act) as Students_enrolled, sum(cap) as Capacity, (sum(cap) - sum(act) )as Remaining_Spots
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE   CatalogYear = "2018_2019" or CatalogYear = "2017_2018" 
GROUP by CatalogYear , program_name 
ORDER BY Students_enrolled dESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


Program,CatalogYear,Students_enrolled,Capacity,Remaining_Spots
Nursing,2017_2018,4999,5337,338
Nursing,2018_2019,4755,5163,408
English,2018_2019,2936,2951,15
English,2017_2018,2880,3068,188
Mathematics,2017_2018,2826,3741,915
Biology,2018_2019,2789,2931,142
Mathematics,2018_2019,2745,3258,513
Biology,2017_2018,2683,2651,-32
Accounting,2017_2018,2392,2260,-132
Accounting,2018_2019,2201,2171,-30


<b> Other Queries:
___

In [60]:
%%sql
SELECT FID, instructor_lname,instructor_fname,CatalogYear,COURSES_FACT.Catalog_ID,program_name, Title ,sum(act), sum(cap), sum(rem) , YEAR,SEMESTER , COUNT(CRN) as COURSESTaught
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE  program_name like "%information%"   and CatalogYear ="2018_2019"
GROUP BY title ,  YEAR,SEMESTER , CatalogYear 

ORDER BY CatalogYear DESC


 * sqlite:///CourseDataWarehouse.db
Done.


FID,instructor_lname,instructor_fname,CatalogYear,Catalog_ID,program_name,Title,sum(act),sum(cap),sum(rem),YEAR,SEMESTER,COURSESTaught
273,Ozcelik,Yasin,2018_2019,IS 0315,Information Systems,Data Mining and Applications,20,20,0,2018,Fall,1
273,Ozcelik,Yasin,2018_2019,IS 0315,Information Systems,Data Mining and Applications,40,40,0,2019,Spring,2
273,Ozcelik,Yasin,2018_2019,IS 0260,Information Systems,Database Systems,20,20,0,2018,Fall,1
273,Ozcelik,Yasin,2018_2019,IS 0260,Information Systems,Database Systems,20,20,0,2019,Spring,1
431,Lee,Patrick S.,2018_2019,IS 0210,Information Systems,Fundamentals of Business Analytics,62,60,-2,2018,Fall,3
431,Lee,Patrick S.,2018_2019,IS 0210,Information Systems,Fundamentals of Business Analytics,45,60,15,2019,Spring,3
274,Vinekar,Vishnu V.,2018_2019,IS 0135,Information Systems,Fundamentals of Web Design,19,20,1,2018,Fall,1
808,Socci,Nicholas F.,2018_2019,IS 0500,Information Systems,Information Systems and Database Management,19,20,1,2018,Fall,1
274,Vinekar,Vishnu V.,2018_2019,IS 0500,Information Systems,Information Systems and Database Management,6,20,14,2019,Spring,1
274,Vinekar,Vishnu V.,2018_2019,IS 0100,Information Systems,Introduction to Information Systems,243,243,0,2018,Fall,9


In [3]:
%%sql
SELECT COUNT(DISTINCT instructor_lname|| '' ||instructor_fname ) as instructor ,COUNT(DISTINCT Title) ,COUNT(Title),sum(act), sum(cap), sum(rem),CatalogYear
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE  program_name like "%information%"  or program_name like "%Business Analytics%"  
GROUP by CatalogYear
ORDER BY CatalogYear DESC;




 * sqlite:///CourseDataWarehouse.db
Done.


instructor,COUNT(DISTINCT Title),COUNT(Title),sum(act),sum(cap),sum(rem),CatalogYear
10,13,39,907,922,15,2018_2019
8,14,36,829,903,74,2017_2018
8,13,34,725,875,150,2016_2017
10,14,34,713,864,151,2015_2016


In [35]:
%%sql
SELECT Title ,CatalogYear ,COURSES_FACT.Catalog_ID , count(COURSES_FACT.Catalog_ID) , sum(act), sum(cap), sum(rem)
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE   CatalogYear = "2018_2019" and program_name like "%information%"  or program_name like "%Business Analytics%"  
GROUP by CatalogYear , Title , COURSES_FACT.Catalog_ID

ORDER BY CatalogYear DESC;

 * sqlite:///CourseDataWarehouse.db
Done.


Title,CatalogYear,Catalog_ID,count(COURSES_FACT.Catalog_ID),sum(act),sum(cap),sum(rem)
Business Analytics,2018_2019,BA0500,1,32,20,-12
Business Intelligence,2018_2019,BA0540,1,30,30,0
Data Mining,2018_2019,BA0545,1,21,30,9
Data Mining and Applications,2018_2019,IS0315,3,60,60,0
Database Systems,2018_2019,IS0260,2,40,40,0
Databases for Business Analytics,2018_2019,BA0510,2,68,64,-4
Fundamentals of Business Analytics,2018_2019,IS0210,6,107,120,13
Fundamentals of Web Design,2018_2019,IS0135,1,19,20,1
Information Systems and Database Management,2018_2019,IS0500,2,25,40,15
Introduction to Information Systems,2018_2019,IS0100,16,424,418,-6


In [11]:
%%sql
SELECT Title ,CatalogYear ,COURSES_FACT.Catalog_ID , count(COURSES_FACT.Catalog_ID) , act, cap, cap - act
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE   program_name like "%Business Analytics%"  and CatalogYear = "2018_2019"
GROUP by CatalogYear , Title , COURSES_FACT.Catalog_ID

ORDER BY CatalogYear DESC;

 * sqlite:///CourseDataWarehouse.db
Done.


Title,CatalogYear,Catalog_ID,count(COURSES_FACT.Catalog_ID),Act,Cap,cap - act
Business Analytics,2018_2019,BA0500,1,32,20,-12
Business Intelligence,2018_2019,BA0540,1,30,30,0
Data Mining,2018_2019,BA0545,1,21,30,9
Databases for Business Analytics,2018_2019,BA0510,2,34,32,-2
Python for Business Analytics,2018_2019,BA0505,1,27,20,-7


In [12]:
%%sql
SELECT CatalogYear ,program_name, sum(act), sum(cap), sum(rem)
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE   CatalogYear = "2018_2019" 
GROUP by CatalogYear , program_name 
HAVING sum(rem) <0
ORDER BY CatalogYear DESC;

 * sqlite:///CourseDataWarehouse.db
Done.


CatalogYear,program_name,sum(act),sum(cap),sum(rem)
2018_2019,Accounting,2201,2171,-30
2018_2019,Business Analytics,178,164,-14
2018_2019,Classical Studies,126,125,-1
2018_2019,Economics,1485,1445,-40
2018_2019,Engineering,412,334,-78
2018_2019,Mechanical Engineering,641,623,-18
2018_2019,Operations Management,433,425,-8
2018_2019,Psychology,1588,1438,-150
2018_2019,Quantitative Analysis,27,20,-7


In [32]:
%%sql

SELECT  program_name, count(*) , count(FID) , sum(Act) ,sum(Cap), sum(Rem)  , YEAR 
FROM COURSES_FACT
left JOIN PROGRAMS_DIM USING (PID)
where program_name  like "%Biology%" 
group by program_name , year
having sum(Rem) <0
order by year desc
    

 * sqlite:///CourseDataWarehouse.db
Done.


program_name,count(*),count(FID),sum(Act),sum(Cap),sum(Rem),YEAR
Biology,160,160,2741,2725,-16,2017
Biology,145,145,2515,2455,-60,2016
Biology,75,75,1304,1200,-104,2015


In [7]:
%%sql
SELECT FID, instructor_lname,instructor_fname,COURSES_FACT.Catalog_ID,program_name, Title ,sum(act), sum(cap), sum(rem) , YEAR,SEMESTER , COUNT(CRN) as COURSESTaught
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE (instructor_lname like '%Huntley%') and year is 2018
GROUP BY title ,  YEAR,SEMESTER  

ORDER BY YEAR DESC


 * sqlite:///CourseDataWarehouse.db
Done.


FID,instructor_lname,instructor_fname,Catalog_ID,program_name,Title,sum(act),sum(cap),sum(rem),YEAR,SEMESTER,COURSESTaught
275,Huntley,Christopher L.,IS0260,Information Systems,Database Systems,24,25,1,2018,Spring,1
275,Huntley,Christopher L.,OM0101,Operations Management,Operations Management,49,50,1,2018,Fall,2
275,Huntley,Christopher L.,BA0505,Business Analytics,Python for Business Analytics,27,20,-7,2018,Fall,1
275,Huntley,Christopher L.,IS0240,Information Systems,Systems Analysis and Logical Design,22,25,3,2018,Spring,1
275,Huntley,Christopher L.,IS0320,Information Systems,Systems Design and Implementation,20,25,5,2018,Spring,1


In [11]:
%%sql
SELECT FID, instructor_lname,instructor_fname,COURSES_FACT.Catalog_ID,program_name, Title ,sum(act), sum(cap), sum(rem) , YEAR,SEMESTER , COUNT(CRN) as COURSESTaught
FROM FACULTY_DIM
    left JOIN COURSES_FACT USING (FID)
     left JOIN COURSE_CATALOGS_DIM USING (CID)
    left  JOIN PROGRAMS_DIM USING (PID)
WHERE (instructor_lname like '%Huntley%') and YEAR IS 2018
GROUP BY title ,  YEAR,SEMESTER 

ORDER BY YEAR DESC


 * sqlite:///CourseDataWarehouse.db
Done.


FID,instructor_lname,instructor_fname,Catalog_ID,program_name,Title,sum(act),sum(cap),sum(rem),YEAR,SEMESTER,COURSESTaught
275,Huntley,Christopher L.,IS0260,Information Systems,Database Systems,24,25,1,2018,Spring,1
275,Huntley,Christopher L.,OM0101,Operations Management,Operations Management,49,50,1,2018,Fall,2
275,Huntley,Christopher L.,BA0505,Business Analytics,Python for Business Analytics,27,20,-7,2018,Fall,1
275,Huntley,Christopher L.,IS0240,Information Systems,Systems Analysis and Logical Design,22,25,3,2018,Spring,1
275,Huntley,Christopher L.,IS0320,Information Systems,Systems Design and Implementation,20,25,5,2018,Spring,1


In [14]:
# do we have  enough sections for students

In [23]:
%%sql

SELECT  Title , count(*) , count(FID) , sum(Act) ,sum(Cap), sum(Rem)  , YEAR 
FROM COURSES_FACT
left JOIN PROGRAMS_DIM USING (PID)
WHERE  Catalog_ID like "%IS%" or   Catalog_ID  like "BA%"
group by Title , YEAR 
having sum(Rem) <0
order by year desc
    

 * sqlite:///CourseDataWarehouse.db
Done.


Title,count(*),count(FID),sum(Act),sum(Cap),sum(Rem),YEAR
Databases for Business Analytics,2,2,68,64,-4,2019
Introduction to Information Systems,7,7,181,175,-6,2019
Business Analytics,1,1,32,20,-12,2018
Data Mining and Applications,2,2,47,45,-2,2018
Python for Business Analytics,1,1,27,20,-7,2018
Fundamentals of Business Analytics,2,2,55,50,-5,2017
Python for Business Analytics,1,1,28,20,-8,2017
Contemporary Topics: Business Analytics with Spreadsheets,1,1,1,0,-1,2016
Fundamentals of Web Design,1,1,26,25,-1,2014
Intro to Information Systems,10,10,237,227,-10,2014


In [13]:
%%sql

SELECT *
FROM COURSES_FACT
JOIN PROGRAMS_DIM USING (PID)
WHERE YEAR = 2017 and program_name like "%information%"
group by Title 


 * sqlite:///CourseDataWarehouse.db
Done.


CID,CRN,SEMESTER,YEAR,Title,SECTION,Act,Cap,Rem,timecodes,PID,LID,FID,Catalog_ID,program_code,program_name
6101,32108,Spring,2017,Business Analytics and Big Data Mgmt,01,17,25,8,"['R 0630pm-0930pm 03/08-04/26 DSB 114', 'S 0830am-0430pm 03/25-03/25 DSB 107', 'S 0830am-0430pm 04/08-04/08 DSB 107', 'S 0830am-0430pm 04/29-04/29 DSB 107']",52,92,619,IS0550,IS,Information Systems
6100,32107,Spring,2017,Data Mining and Business Intelligence,01,19,25,6,['W 0630pm-0930pm 01/17-05/12 DSB 114'],52,92,619,IS0540,IS,Information Systems
923,71341,Fall,2017,Database Systems,01,23,25,2,['TF 0330pm-0445pm 09/05-12/11 DSB 112'],52,7,273,IS0260,IS,Information Systems
1152,72914,Fall,2017,Databases for Business Analytics,01,18,20,2,"['T 0630pm-0930pm 10/31-12/12 DSB 115', 'S 0830am-0430pm 11/04-11/04 DSB 115', 'S 0830am-0430pm 11/18-11/18 DSB 115', 'S 0830am-0430pm 12/09-12/09 DSB 115']",52,9,275,IS0510,IS,Information Systems
924,71287,Fall,2017,E-Business Applications,01,19,20,1,['W 1100am-0130pm 09/05-12/11 DSB 2109A'],52,10,274,IS0310,IS,Information Systems
6092,31734,Spring,2017,Fundamentals of Business Analytics,01,23,25,2,['MR 0930am-1045am 01/17-05/02 CNS 301'],52,68,431,IS0210,IS,Information Systems
920,71929,Fall,2017,Fundamentals of Web Design,01,17,20,3,['MR 0330pm-0445pm 09/05-12/11 DSB 109'],52,8,274,IS0135,IS,Information Systems
6098,32651,Spring,2017,Information Systems and Database Management,01,15,25,10,"['R 0630pm-0930pm 01/17-03/02 DSB 115', 'S 0830am-0430pm 01/28-01/28 DSB 115', 'S 0830am-0430pm 02/11-02/11 DSB 115', 'S 0830am-0430pm 02/25-02/25 DSB 115']",52,9,619,IS0500,IS,Information Systems
857,71166,Fall,2017,Introduction to Information Systems,01,30,28,-2,['TF 0930am-1045am 09/05-12/11 DSB 105'],52,1,273,IS0100,IS,Information Systems
6099,32647,Spring,2017,Project Management,01,14,25,11,"['T 0630pm-0930pm 03/07-04/25 DSB 105', 'S 0830am-0430pm 03/25-03/25 DSB 105', 'S 0830am-0430pm 04/08-04/08 DSB 105', 'S 0830am-0430pm 04/22-04/22 DSB 105']",52,1,278,IS0520,IS,Information Systems


In [29]:
%%sql
vacuum; 

 * sqlite:///CourseDataWarehouse.db
Done.


[]